In [9]:
import mysql.connector
import mysql.connector as connector
import re
import datetime


class FoodOrederDB:
    connetdb = []

    def __init__(self):
        self.connetdb = connector.connect(host='localhost', user="root", password="shiva1610", database="FoodOrderDB")
        self.mycursor = self.connetdb.cursor()
        print("connect server")

    """
    def showDB(self):
        self.mycursor = self.connetdb.cursor()
        self.mycursor.execute("SHOW DATABASES")
        for x in self.mycursor:
            print(x)
        print("DATABASES")
    """

    def restuarnt_menu(self):
        restuarnt_input = input(""" WELLCOME TO EDYODA RESTUARNT
        Specify correct user type
        For Admin = Type "Admin"
        For new user =  Type "Register"
        For old user = Type "Login"
        Enter anything for exit \n""")
        if restuarnt_input == "Login":
            self.login_menu()
        elif restuarnt_input == "Register":
            self.register_menu()
        elif restuarnt_input == "Admin":
            self.admin_menu()
        else:
            print("Please enter correct user type....")
            self.homepage()

    def homepage(self):
        Home_input = input("For Goto homepage Y/N --- ")
        if Home_input == "Y":
            self.restuarnt_menu()
        else:
            print("--- Thank you for Visiting ---")


#ADMIN LOGIN MENU START HERE
    def admin_menu(self):
        print("---ENTER ADMIN DETAILS--")
        self.admin = str(input("Type 'Admin' Username : "))
        self.adminpassword = (input("Enter Admin Password : "))
        if self.admin == "Admin" and self.adminpassword == "Admin123":
            print("Admin Succesfully login")
            self.food_menu()

        else:
            print("May be Userame or Password are wrong")
            self.homepage()

    def food_menu(self):
        print("""WELL COME TO EDYODA RESTURANT ADMIN PANEL
        For Adding Food -- Type "Add"
        For Edit the items -- Type "Edit"
        For View the food list -- Type "View"
        For View the Order Histrory list -- Type "HistoryView"
        For Remove the food from food list -- Type "Remove"
        Enter anything for exit \n""")
        admin_input = input("Specify correct activity you want to do --- ")
        if admin_input == "Add":
            self.Add_food()
        elif admin_input == "Edit":
            self.Edit_food()
        elif admin_input == "View":
            self.View_food()
        elif admin_input == "HistoryView":
            self.history_menu_admin()
        elif admin_input == "Remove":
            self.Remove_food()
        else:
            self.homepage()

    def Add_food(self):
        self.Fname = str(input("Enter Food Name : "))
        self.Ftype = str(input("Enter Veg or NonVeg type : "))
        self.Fquantity = str(input("Enter quantity of food : "))
        self.Fprice = int(input("Enter price/quatity : "))
        self.Fdiscount = float(input("Enter discount amount in % : "))
        self.Fstock = int(input("Enter intial Stock : "))
        self.Fstockleft = 0 + self.Fstock
        # self.mycursor.execute("drop table if exists foodlist")
        foodlist = "create table if not exists foodlist( FId int(4) not null auto_increment primary key ,Fname varchar(20) not null,Ftype varchar(20) not null, Fquantity varchar(20) not null, Fprice int(5) not null,Fdiscount int(3) not null, Fstock int(5) not null,Fstockleft int(5) not null);"
        self.mycursor.execute(foodlist)
        """self.mycursor.execute('desc foodlist')
        for x in self.mycursor:
            print(x)"""
        fooddata = "insert into foodlist(FId, Fname, Ftype, Fquantity, Fprice,Fdiscount ,Fstock,Fstockleft) values (null, %s,%s,%s,%s,%s,%s,%s)"
        foodvalue = (self.Fname, self.Ftype, self.Fquantity, self.Fprice, self.Fdiscount, self.Fstock, self.Fstockleft)
        self.mycursor.execute(fooddata, foodvalue)
        self.connetdb.commit()
        self.mycursor.execute("select FId from foodlist")
        myresult = self.mycursor.fetchall()
        show_food_id = myresult[-1]
        print("Please Note Food ID is", (show_food_id))

        food_y=input("you want continune to add food Y/N -- ")
        if food_y=="Y":
            self.Add_food()
        else:
            self.food_menu()

    def View_food(self):
        self.mycursor.execute("select * from foodlist")
        print("FOOD MENU")
        print("FID,FNAME,FTYPE,FQUANTITY,FPRICE,FDISCOUNT,FINTIALSTOCK,FSTOCKLEFT")
        for x in self.mycursor:
            print(x)
        st_input = input("Goto to admin Y/N ")
        if st_input == "Y":
            self.food_menu()
        else:
            self.homepage()

    def history_menu_admin(self):
        OId = input("Enter Order Id to know the status : ")
        self.mycursor.execute("select * from orderhistory where OId = {}".format(OId))
        for x in self.mycursor:
            print(x)
        st_input = input("Goto to admin Y/N ")
        if st_input == "Y":
            self.food_menu()
        else:
            self.homepage()

    def Remove_food(self):
        self.View_food()
        data1 = input("Enter the FId you want remove : ")
        sql = "DELETE FROM foodlist WHERE FId = {}".format(data1)
        self.mycursor.execute(sql)
        self.connetdb.commit()
        print("Food item succesfully removed from food list")

        food_y = input("you want continune to Remove food Y/N -- ")
        if food_y == "Y":
            self.Remove_food()
        else:
            self.food_menu()


    def Edit_food(self):
        st_in = input(print("You want to food menu Y/N --- "))
        if st_in == "Y":
            self.View_food()
        else:
            
            print("FId, Fname, Ftype, Fquantity, Fprice,Fdiscount ,Fstock,Fstockleft")
            data2 = input("Enter FId --- ")
            column = input("Enter column name --- ")
            data1 = input("Enter new data --- ")
            sql = "UPDATE foodlist SET {} = {} WHERE FId = {}".format(column, data1, data2)
            self.mycursor.execute(sql)
            self.connetdb.commit()
            print("Food item succesfully updated ")

            food_y = input("you want continune to Upadated food Y/N -- ")
            if food_y == "Y":
                self.Edit_food()
            else:
                self.food_menu()

# REGISTATION MENU START HERE
    def register_menu(self):
        userlist = "create table if not exists userlist( UId int(10) not null auto_increment primary key ,Uname varchar(258) not null, Uage int(3) not null,Umail varchar(50) not null,Uphnum varchar(10) not null, Uwallet int(5) not null ,Uadd varchar(250) not null,Upassword int(8) not null);"
        self.mycursor.execute(userlist)
        self.connetdb.commit()
        print("Enter your user details here")
        self.Uname = str(input("Enter User Name ---  "))
        self.Uage = input("Enter User Age --- ")
        self.Umail1 = str(input("Enter Your Email --- "))
        regex = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b'
        if(re.fullmatch(regex, self.Umail1)):
            self.Umail=self.Umail1
        else:
            print("---Enter valid Email ID should be @gmail.com---")
            self.register_menu()

        self.Uphnum1 = (input("Enter Phone Number --- "))
        self.mycursor.execute("select count(*) UId from userlist where Uphnum like {}".format(self.Uphnum1))
        uid1 = self.mycursor.fetchall()
        for x1 in uid1:
            for y in x1:
                self.ul = y
        regx = r'^[0-9]{10}$'
        if (re.fullmatch(regx, self.Uphnum1)):
            self.Uphnum = self.Uphnum1
            if self.ul > 0:
                print("Phone number is already exist")
                self.register_menu()

        else:
            print("---Enter valid Phone Number should be 10 digit---")
            self.register_menu()

        self.Uadd = input("Enter Your Address --- ")
        Upassword1 = input("Enter Password --- ")
        Upassword2 = input("Confirm your password --- ")
        if Upassword1 == Upassword2:
            self.Upassword = Upassword2
            self.user_list()
        else:
            print("Password mismatch please enter details corectly")
            self.register_menu()

#ADDING USER DETAILS TO USER TABLE
    def user_list(self):
        #self.mycursor.execute("trucate userlist")
        userlist = "create table if not exists userlist( UId int(10) not null auto_increment primary key ,Uname varchar(258) not null, Uage int(3) not null,Umail varchar(50) not null,Uphnum varchar(10) not null, Uwallet int(5) not null ,Uadd varchar(250) not null,Upassword int(8) not null);"
        self.mycursor.execute(userlist)
        self.Uwallet = 0
        userdata = "insert into userlist(UId, Uname, Uage, Umail, Uphnum, Uwallet,Uadd,Upassword) values (null,%s, %s,%s,%s,%s,%s,%s)"
        uservalue = (self.Uname, self.Uage, self.Umail, self.Uphnum,self.Uwallet, self.Uadd, self.Upassword)
        self.mycursor.execute(userdata, uservalue)
        self.connetdb.commit()
        self.mycursor.execute("select UId from userlist")
        myresult = self.mycursor.fetchall()
        show_user_id = myresult[-1]

        print("Please Note Your User ID --- ", str(show_user_id))

        Uwallet1 = int(input("Enter wallet amount --- "))
        sql='update userlist set Uwallet = {} where UId = 1'.format(Uwallet1,show_user_id)
        self.mycursor.execute(sql)

        self.connetdb.commit()
        self.homepage()






#USER LOGIN MENU START HERE
    def login_menu(self):
        print("Enter your User ID and Password")
        self.userId = input("Enter your user Id : ")
        self.userpassword = input("Enter your password : ")
        self.mycursor.execute("select * from userlist where UId like {} AND Upassword like {}".format(self.userId, self.userpassword))
        userlist = self.mycursor.fetchall()
        if len(userlist) > 0:
            print("Login succesfully")
            self.current_user_id = userlist[0][0]
            print(self.current_user_id)
            self.user_menu()
        else:
            print("""User ID and Password are incorrect details
                  type anything for Exit /n""")
            fo = input("forgtten your password = Type Y/N :")
            if fo == "Y":
                us = input("Enter your phone number : ")
                self.mycursor.execute( "select UId from userlist where Uphnum like {}".format(us))
                uid1 =self.mycursor.fetchall()
                for x1 in uid1:
                    for y in x1:
                        self.ul = y
                        print("Your user ID is -- " ,self.ul)
                    self.mycursor.execute("select Uphnum from userlist where UId = {}".format(self.ul))
                    ph1 = self.mycursor.fetchall()
                    for p1 in ph1:
                        for p2 in p1:
                            self.pl = p2
                    if us == self.pl:
                        self.mycursor.execute("select Upassword from userlist where UId = {}".format(self.ul))
                        for t in self.mycursor:
                            for c in t:
                                print("Your Password is -- ", c)
                                self.homepage()
                else:
                    print("Phone number details are not present")
                    self.homepage()

            elif fo == "N":
                self.login_menu()



    def user_menu(self):
        print("WELLCOME TO EDYODA RESTURANT")
        print("""For placing new odrer -- Type "Order"
              For Order History -- Type "History"
              For Update profile -- Type "Profile" """)
        Uselection = str(input("Enter your selection : "))
        if Uselection == "Order":
            # self.food_order_history()
            self.order_menu()
        elif Uselection == "History":
            self.history_menu()
        elif Uselection == "Profile":
            self.profile_menu()
        else:
            print("Type correct activity type")
            self.user_menu()

    def order_menu(self):
        self.mycursor.execute("select * from foodlist")
        print("FOOD MENU")
        print("FID,FNAME,FTYPE,FQUANTITY,FPRICE,FDISCOUNT,FINTIALSTOCK,FSTOCKLEFT")
        for x in self.mycursor:
            print(x)
        print("Type 'Food Id' number  like 1 2 3 4 -- ")
        self.orderlist = list(map(int, input("Enter elements of array -- ").strip().split()))
        print(self.orderlist)
        confirmFId = (
            input("check and confirm if food Id are correct type 'Procced' or for remove order type 'Remove' --- "))
        if confirmFId == "Procced":
            self.order_confirm_menu()
        elif confirmFId == "Remove":
            print("Type remove 'Food Id' number like 1 2 3 4 -- ")
            orderlistt = list(map(int, input("Enter elements of array:-").strip().split()))
            print(orderlistt)
            for x in orderlistt:
                self.orderlist.remove(x)
            print(self.orderlist)
            self.order_confirm_menu()
        else:
            print("Type correct activity type")
            self.order_menu()

    def order_confirm_menu(self):
        # self.mycursor.execute("drop table if exists orderhistory")
        orderhistory = "create table if not exists orderhistory( OId int(10) not null default 0 ,UId int(10) not null,Odate datetime not null,FId int(4) not null, Fname varchar(20) not null,Fprice int(5) not null, Oquantity int(3) not null,Oprice int(5) not null,Uadd varchar(250) not null);"
        self.mycursor.execute(orderhistory)
        self.mycursor.execute('select OId FROM orderhistory WHERE OId = (SELECT MAX(OId) FROM orderhistory)')
        for j in self.mycursor:
            for u in j:
                self.OId = u + 1
        self.UId = self.current_user_id
        self.mycursor.execute("select Uadd from userlist where UId = {}".format(self.UId))
        for t in self.mycursor:
            for x in t:
                self.Uadd = str(x)
                #print("Your order Location is ---",self.Uadd)
        self.Odate = datetime.datetime.utcnow()
        #print("Your Ordered DATE and TIME is--",self.Odate)
        print("Your Ordered FId array list is -- ",self.orderlist)
        self.Ototalprice = 0
        for i in self.orderlist:
            print(i)
            self.FId = i
            print("Your food order ID is : ", self.FId)
            self.mycursor.execute("select Fname from foodlist  where FId = {}".format(i))
            for y in self.mycursor:
                for f in y:
                    self.Fname = str(f)
                    print("Your oder food name is -- ", self.Fname)
            self.mycursor.execute("select Fprice from foodlist  where FId = {}".format(i))
            for x in self.mycursor:
                for y in x:
                    self.Fprice = y
                print("Food amount is -- ", self.Fprice)
            self.mycursor.execute("select Fdiscount from foodlist  where FId = {}".format(i))
            for t in self.mycursor:
                for h in t:
                    self.Fdiscount = h
                print("Your discount amount is in % -- ", self.Fdiscount)
                self.Oquantity = int(input("Enter Quantity you want to order -- "))
                self.Sleft()
                self.Oprice = self.Oquantity * (self.Fprice - (self.Fdiscount / 100 * self.Fprice))
                print("Your Food ID {} and Food Name {} ammount is {} ".format(self.FId, self.Fname, self.Oprice))
                self.Ototalprice += self.Oprice
            print("Your order Location is ---", self.Uadd)
            print("Your Ordered DATE and TIME is--", self.Odate)
            print("Your total order amount is after all discount", self.Ototalprice)
            orderdata = "insert into orderhistory(OId, UId, Odate,FId,Fname,Fprice, Oquantity,Oprice ,Uadd) values (%s,%s,%s,%s,%s,%s,%s,%s,%s)"
            ordervalue = (
            self.OId, self.UId, self.Odate, self.FId, self.Fname, self.Fprice, self.Oquantity, self.Oprice, self.Uadd)
            self.mycursor.execute(orderdata, ordervalue)
            self.connetdb.commit()
            self.mycursor.fetchall()
            self.Sleft()
        confirm1 = input("""If you want place order type = "Confirm"
                    and for cancel order type = "Cancel" : """)
        if confirm1 == "Confirm":
            self.wallet_check()
            self.homepage()
        else:
            self.Ocancel()
            self.homepage()

    def Oconfirm1(self):
            self.mycursor.execute("select OId from orderhistory")
            myresult = self.mycursor.fetchall()
            self.show_order_id = myresult[-1]
            print("Please Note Ordered ID is", (self.show_order_id))

    def Ocancel(self):
            self.mycursor.execute("select OId from orderhistory")
            myresult = self.mycursor.fetchall()
            self.show_order_id = myresult[-1]
            for i in self.show_order_id:
                x = i
            self.mycursor.execute("delete from orderhistory where OId={}".format(x))
            self.connetdb.commit()
            self.mycursor.fetchall()
    
    def wallet_check(self):
        self.mycursor.execute("select Uwallet from userlist where UId = {}".format(self.current_user_id))
        myresult = self.mycursor.fetchall()
        show_user_id = myresult
        for x in show_user_id:
            for y in x:
                self.waamt=y
        if self.waamt > self.Ototalprice:
            self.wabal= self.waamt-self.Ototalprice
            self.mycursor.execute("UPDATE userlist SET Uwallet = {} WHERE UId = {}".format(self.wabal, self.current_user_id))
            myresult1 = self.mycursor.fetchall()
            self.mycursor.execute("select Uwallet from userlist where UId = {}".format(self.current_user_id))
            myresult = self.mycursor.fetchall()
            show_user_id = myresult
            for x in show_user_id:
                for y in x:
                    print("Your Wallet Balanace Amount is --- ",y)
                    self.Oconfirm1()

        else:
            print("Insufficient balance")
            adamt=input("Do you want add the amount to wallet Y/N --- ")
            if adamt == "Y":
                adamt1 = int(input("Enter amount -- ")) + self.waamt
                self.mycursor.execute( "UPDATE userlist SET Uwallet = {} WHERE UId = {}".format(adamt1, self.current_user_id))
                self.mycursor.fetchall()
                self.mycursor.execute("select Uwallet from userlist where UId = {}".format(self.current_user_id))
                myresult = self.mycursor.fetchall()
                print(myresult)
                show_user_id = myresult
                for x in show_user_id:
                    for y in x:
                        print("Your Wallet Balanace Amount is --- ", y)
                        self.wallet_check()

            else:
                self.Ocancel()

    def Sleft(self):
        self.mycursor.execute("select Fstockleft from foodlist where FId = {}".format(self.FId))
        myresult = self.mycursor.fetchall()
        show_food_stock = myresult
        for x in show_food_stock:
            for y in x:
                self.stockleft = y
                #print("Stock left",y)
        if self.stockleft > self.Oquantity:
            self.stbal= self.stockleft-self.Oquantity
            self.mycursor.execute("UPDATE foodlist SET Fstockleft = {} WHERE FId = {}".format(self.stbal, self.FId))
            myresult1 = self.mycursor.fetchall()
            self.mycursor.execute("select Fstockleft from foodlist where FId = {}".format(self.FId))
            myresult = self.mycursor.fetchall()
            show_food_stock = myresult
            #for x in show_food_stock:
             #   for y in x:
                  #print("Stock Balanace  is --- ",y)

        else:
            print("Out of Stock")
            print("only {} stocks are available".format(self.stockleft))
            adamt=input("Do you want add new quantity to order for continue Y/N --- ")
            if adamt == "Y":
                self.Oquantity = int(input("Enter new order quantity -- "))
            else:
                self.Ocancel()

    def history_menu(self):
            self.mycursor.execute("select * from orderhistory where UId = {}".format(self.current_user_id))
            for x in self.mycursor:
                print(x)

    def profile_menu(self):
            UId = self.current_user_id
            print("UId, Uname, Uage, Umail, Uphnum, Uwallet,Uadd,Upassword")
            self.mycursor.execute("select * from userlist where UId = {}".format(UId))
            for x in self.mycursor:
                print(x)
            t1 = str(input("Enter coloumn want you want update - "))
            d1 = str(input("Enter you new data you want update - "))
            self.mycursor.execute("UPDATE userlist SET {} = '{}' WHERE UId = UId".format(t1, d1))
            self.mycursor.execute("select * from userlist where UId = {}".format(UId))
            for x in self.mycursor:
                print(x)
            self.connetdb.commit()


food = FoodOrederDB()
food.restuarnt_menu()

connect server
 WELLCOME TO EDYODA RESTUARNT
        Specify correct user type
        For Admin = Type "Admin"
        For new user =  Type "Register"
        For old user = Type "Login"
        Enter anything for exit 
Admin
---ENTER ADMIN DETAILS--
Type 'Admin' Username : Admin
Enter Admin Password : Admin123
Admin Succesfully login
WELL COME TO EDYODA RESTURANT ADMIN PANEL
        For Adding Food -- Type "Add"
        For Edit the items -- Type "Edit"
        For View the food list -- Type "View"
        For View the Order Histrory list -- Type "HistoryView"
        For Remove the food from food list -- Type "Remove"
        Enter anything for exit 

Specify correct activity you want to do --- Add
Enter Food Name : Egg Biriyani 
Enter Veg or NonVeg type : Nonveg
Enter quantity of food : 1 plate
Enter price/quatity : 100
Enter discount amount in % : 5
Enter intial Stock : 60
Please Note Food ID is (13,)
you want continune to add food Y/N -- N
WELL COME TO EDYODA RESTURANT ADMIN PANE